## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
import numpy as np

# Import API key
sys.path.insert(0, '/users/peyton/World_Weather_Analysis')
from gconfig import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/users/peyton/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Raga,SS,8.4596,25.6780,68.36,25,63,3.24,broken clouds
1,1,Kirakira,SB,-10.4544,161.9205,85.33,73,81,9.53,broken clouds
2,2,Mataura,NZ,-46.1927,168.8643,56.48,45,74,7.02,broken clouds
3,3,Nizhniy Tsasuchey,RU,50.5167,115.1333,-8.66,99,92,1.54,overcast clouds
4,4,Yongan,CN,39.6957,113.6871,21.72,89,7,1.54,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Kirakira,SB,-10.4544,161.9205,85.33,73,81,9.53,broken clouds
5,5,Sao Filipe,CV,14.8961,-24.4956,74.64,74,89,4.70,overcast clouds
7,7,East London,ZA,-33.0153,27.9116,72.66,71,0,18.07,clear sky
9,9,Kapaa,US,22.0752,-159.3190,76.98,77,100,1.01,overcast clouds
10,10,Port Alfred,ZA,-33.5906,26.8910,72.37,74,0,26.75,clear sky
18,18,Puerto Suarez,BO,-18.9500,-57.8000,89.64,43,80,6.17,broken clouds
19,19,Port Elizabeth,ZA,-33.9180,25.5701,72.10,83,0,9.22,clear sky
21,21,Butaritari,KI,3.0707,172.7902,82.83,71,92,12.62,overcast clouds
23,23,Busselton,AU,-33.6500,115.3333,71.10,46,4,6.73,clear sky
24,24,Faanui,PF,-16.4833,-151.7500,81.30,76,100,16.33,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna(how="any")

In [7]:
# Check to see if there are any remaining null rows
preferred_cities_clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kirakira,SB,85.33,broken clouds,-10.4544,161.9205,
5,Sao Filipe,CV,74.64,overcast clouds,14.8961,-24.4956,
7,East London,ZA,72.66,clear sky,-33.0153,27.9116,
9,Kapaa,US,76.98,overcast clouds,22.0752,-159.3190,
10,Port Alfred,ZA,72.37,clear sky,-33.5906,26.8910,
18,Puerto Suarez,BO,89.64,broken clouds,-18.9500,-57.8000,
19,Port Elizabeth,ZA,72.10,clear sky,-33.9180,25.5701,
21,Butaritari,KI,82.83,overcast clouds,3.0707,172.7902,
23,Busselton,AU,71.10,clear sky,-33.6500,115.3333,
24,Faanui,PF,81.30,light rain,-16.4833,-151.7500,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        pass
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how="any")

In [14]:
# Check to see if there are any NA values remaining.
clean_hotel_df.isna().count()

City                   258
Country                258
Max Temp               258
Current Description    258
Lat                    258
Lng                    258
Hotel Name             258
dtype: int64

In [12]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kirakira,SB,85.33,broken clouds,-10.4544,161.9205,SanBiz Lodge
5,Sao Filipe,CV,74.64,overcast clouds,14.8961,-24.4956,Hotel Xaguate
7,East London,ZA,72.66,clear sky,-33.0153,27.9116,Tu Casa
9,Kapaa,US,76.98,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
10,Port Alfred,ZA,72.37,clear sky,-33.5906,26.8910,The Halyards Hotel
18,Puerto Suarez,BO,89.64,broken clouds,-18.9500,-57.8000,Hotel casa real
19,Port Elizabeth,ZA,72.10,clear sky,-33.9180,25.5701,39 On Nile Guest House
21,Butaritari,KI,82.83,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
23,Busselton,AU,71.10,clear sky,-33.6500,115.3333,Observatory Guest House
24,Faanui,PF,81.30,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))